In [33]:
import os
import torch
import pickle
from transformers import BertTokenizer, BertModel, CamembertModel, CamembertTokenizer, AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Avec CamemBERT (pas au point)

In [136]:
def get_document_embeddings_batch(texts, model, tokenizer, df):
    vectors = []
    for i, text in enumerate(df.text):
        print(f"\r{i} / {len(df)}", end="")
        input_ids = tokenizer(text, return_tensors="pt", truncation = True, padding = True)
        with torch.no_grad():
            outputs = model(**input_ids)
        vectors.append(outputs.last_hidden_state.mean(dim = 1))
    return(torch.cat(vectors, dim=0))

def semantic_search(query, model, tokenizer, document_embeddings, n_results=5):
    query_embedding = get_document_embeddings_batch(query, model, tokenizer)
    similarity_scores = cosine_similarity(query_embedding, document_embeddings)[0]
    return(similarity_scores)
    # sorted_indices = similarity_scores.argsort()
    # return(sorted_indices[::-1][:n_results])

def save_embeddings(embeddings, output_file):
    with open(output_file, 'wb') as f:
        pickle.dump(embeddings, f)

def load_embeddings(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)
    
def sortfunc(x):
    return(int(x))

In [135]:

if __name__ == "__main__":
    # Replace 'bert-base-uncased' with any other suitable pre-trained model if needed.
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # model = BertModel.from_pretrained('bert-base-uncased')
    modelname = 'camembert/camembert-base'
    model = AutoModel.from_pretrained("camembert-base")
    tokenizer = AutoTokenizer.from_pretrained("camembert-base")
    
    MODE = "Update"
    
    embeddings_file = "document_embeddings.pkl"

    data_folder = "./data/Texte/"
    n_results = 5  # Change this value to get a different number of relevant documents.

    # Load and process your text data from separate folders.
    documents = []
    index = []
    
    listdir = list(os.listdir("./data/Texte/"))
    listdir.sort(key=sortfunc)
    
    for folder_name in listdir:
        print(f"\r{folder_name}", end = "")
        folder_path = os.path.join(data_folder, folder_name)
        if not os.path.isdir(folder_path):
            continue
        for file_name in os.listdir(folder_path):
            if file_name == "full.txt" :
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()
                    documents.append(text)
                    index.append(int(folder_name))
    
    df = pd.DataFrame(documents, columns = ["text"])
    df.index = index
    # Get embeddings for all the documents.
    if os.path.exists(embeddings_file) and MODE != "Update":
        print("\nLOADING EMBEDDINGS")
        document_embeddings = load_embeddings(embeddings_file)
    else:
        print("\nCONVERTING DOCUMENTS INTO EMBEDDINGS")
        document_embeddings = get_document_embeddings_batch(documents, model, tokenizer, df)
        save_embeddings(document_embeddings, embeddings_file)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2784CONVERTING DOCUMENTS INTO EMBEDDINGS


ValueError: Expected a 1D array, got an array with shape (2774, 768)

In [134]:
document_embeddings

tensor([[ 0.0420,  0.0376, -0.0085,  ..., -0.0484,  0.0512,  0.0441],
        [ 0.0593,  0.0805, -0.0157,  ..., -0.0258,  0.0395,  0.0781],
        [ 0.0485, -0.0184, -0.0465,  ..., -0.0457,  0.0394,  0.0570],
        ...,
        [ 0.0398, -0.0043, -0.0474,  ..., -0.0452,  0.0478,  0.0528],
        [ 0.0383, -0.0159, -0.0229,  ..., -0.0343,  0.0216,  0.0570],
        [ 0.0424, -0.0213, -0.0600,  ..., -0.0345,  0.0487,  0.0616]])

In [63]:
# Example query.
query = "manger, faim, bouffe"

# Get the most relevant documents.
relevant_indices  = semantic_search(query, model, tokenizer, document_embeddings.vectors, n_results)
# Print the most relevant documents.
res = pd.DataFrame(zip(documents, relevant_indices), columns = ["Doc", "Cos"]).sort_values(by="Cos", ascending=False)
res.head(20)
# print("Most relevant documents:")
# for i, index in enumerate(relevant_indices):
#     print(documents[index])
#     print("------------------------")

,Doc,Cos
906,Petit lexique des fleuves\nnan\n,0.668709
157,1-test ressource\nnan\nnan,0.642991
368,La longue marche de solid’arte\nnan\nArrêt sur...,0.611183
1141,Ritualités et transmission : le symptôme commé...,0.536674
381,La sécurité urbaine : un bien public\nnan\nDan...,0.505748
986,"Moncey, pour un aménagement adapté\nnan\nTexte...",0.498714
1045,L’Etat au coeur : le meccano de la gouvernance...,0.488320
204,Acquérir de nouvelles compétences : quelques p...,0.476472
347,"Lyon, métropole de la lumière\nnan\nUsages et ...",0.475257
388,Deux exemples de stratégie hospitalière visant...,0.471568


In [68]:
res.Doc[50]

'Le marché du tourisme urbain dans les métropoles françaises\nAtout France a pour missions de promouvoir et développer la marque France à l’international, d’adapter l’oﬀre française à la demande touristique nationale et internationale, d’accompagner les acteurs privés et publics du secteur touristique.Dans cette interview, Sophie Mandrillon et Marie-Laure Desmet évoquent les efforts engagés par les métropoles françaises, et notamment Lyon, pour se positionner sur le marché européen du tourisme urbain.\n\n\nOn sait que le tourisme urbain est en plein essor dans le monde. Quel regard portez-vous sur le marché mondial aujourd’hui\xa0?Les destinations urbaines constituent effectivement de puissants moteurs de croissance du tourisme dans le monde, à la fois comme émettrices et comme réceptrices de flux touristiques. Elles concentrent une activité économique, administrative, universitaire, une offre de commerce et de loisirs et un patrimoine culturel qui leur permettent de capter une grande 

# AVEC TF-IDF

In [156]:
from sklearn.feature_extraction.text import CountVectorizer

data_folder = "./data/Texte/"
documents=[]
nom_docs=[]
for folder_name in os.listdir(data_folder):
    # print(folder_name)
    folder_path = os.path.join(data_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue
    for file_name in os.listdir(folder_path):
        # print(file_name)
        if file_name == "texte.txt":
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
                documents.append(text)
                nom_docs.append(folder_path.split('/')[3])
                # print(folder_path.split('/')[3], text)
vectorizer=CountVectorizer(lowercase=False)
nOcc=vectorizer.fit_transform(documents)
nOcc=pd.DataFrame.sparse.from_spmatrix(nOcc,index=nom_docs,columns=vectorizer.get_feature_names_out())

In [172]:
query = vectorizer.transform(["musique art culture"]) #Process (lemmatize + stopwords)

similarities = cosine_similarity(query, nOcc)

# Classement des documents en fonction de leur similarité avec la requête
sorted_indices = similarities[0]
sim = pd.DataFrame(sorted_indices, nom_docs).sort_values(by=0, ascending=False)

In [185]:
txt = []
for doc in sim[:5].index:
    with open(f"./data/Texte/{doc}/full.txt", "r", encoding="utf-8") as f:
        texte = f.read()
        print(texte)
        txt.append(texte)
    print("----------------------")

Lyon et la question des « arts »
nan
Interroger les singularités de l'identité culturelle lyonnaise ne peut se faire sans que l'on revisite les différentes acceptions qu'a pu revêtir le terme " art ". Le terme a été référé initialement au champ de l'utilité? bien avant de désigner celui de la spéculation intellectuelle ou celui du beau et du désintéressement. La question se pose de savoir si, à l'orée du 21e siècle, Lyon peut mettre ses institutions au service d'une actualisation des trois sens du terme.Sommaire :1 Pourquoi Lyon n’a pu servir la cause des «  arts libéraux »Mais tout d’abord qu’appelle-t-on « arts libéraux » ?Pourquoi Lyon ne s’est-elle pas illustrée dans les « arts libéraux » ?2 Pourquoi Lyon a remarquablement servi la cause des « arts mécaniques », mais sur un mode éminemment paradoxal Les « arts  mécaniques, arts de transformation de la matièreMais un double paradoxe ne peut pas ne pas être relevéLe contre-exemple nancéen3 Pourquoi Lyon a servi la cause des « beaux-a

### Resumé avec ChatGPT

In [194]:
import openai

with open("APIKEY", "r", encoding="utf-8") as f:
    openai.api_key = f.read()

prompt = f"""Je cherche à synthétiser au mieux l'information. Résume l'ensemble de ces différents textes séparés par ce signe "---" en très peu de phrases : {"---".join(txt)}"""

openai.organization = "org-mqd2akvmI5TNokavyM3mWXSq"
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages = [{"role": "user", 
                 "content": prompt}]
    )
print(chat_completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Lyon a connu une \u00e9volution dans sa vision des arts, passant d'une utilisation limit\u00e9e dans le champ de l'utilit\u00e9 \u00e0 une reconnaissance de l'importance des arts m\u00e9caniques, des beaux-arts et de leur r\u00f4le dans l'enseignement. Cependant, il reste encore des limites \u00e0 l'expansion de l'art dans l'enseignement, notamment dans l'enseignement sup\u00e9rieur o\u00f9 les disciplines restent cloisonn\u00e9es, et les politiques d'art public sont encore timides dans l'agglom\u00e9ration lyonnaise malgr\u00e9 quelques initiatives.",
        "role": "assistant"
      }
    }
  ],
  "created": 1689850633,
  "id": "chatcmpl-7eLdJggwqVaiWoAkYv8re1zmNigA7",
  "model": "gpt-3.5-turbo-16k-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 132,
    "prompt_tokens": 9168,
    "total_tokens": 9300
  }
}


In [195]:
print(chat_completion["choices"][0]["message"]["content"])


Lyon a connu une évolution dans sa vision des arts, passant d'une utilisation limitée dans le champ de l'utilité à une reconnaissance de l'importance des arts mécaniques, des beaux-arts et de leur rôle dans l'enseignement. Cependant, il reste encore des limites à l'expansion de l'art dans l'enseignement, notamment dans l'enseignement supérieur où les disciplines restent cloisonnées, et les politiques d'art public sont encore timides dans l'agglomération lyonnaise malgré quelques initiatives.


# Pipeline avec LLama2 & HuggingFace